In [1]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [2]:
with open("../war_and_peace.txt", "r", encoding="utf-8") as f:
    text = f.read()
    
print(len(text))

print("Total Characters", len(text))

print(text[:1000])

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

3227518
Total Characters 3227518
The Project Gutenberg eBook of War and Peace, by Leo Tolstoy

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: War and Peace

Author: Leo Tolstoy

Translators: Louise and Aylmer Maude

Release Date: April, 2001 [eBook #2600]
[Most recently updated: June 14, 2022]

Language: English

Character set encoding: UTF-8

Produced by: An Anonymous Volunteer and David Widger

*** START OF THE PROJECT GUTENBERG EBOOK WAR AND PEACE ***




WAR AND PEACE


By Leo Tolstoy/Tolstoi


    Contents

    BOOK ONE: 1805

    CHAPTER I

    CHAPTER II

    CHAPTER III

    CHAPTE

In [12]:
import os

print("CWD:", os.getcwd())
print("Exists:", os.path.exists("generatively_pretrained_transformer/war_and_peace.txt"))

CWD: /Users/anthonylewallen/Programming/ai_engineering/generatively_pretrained_transformer/src
Exists: False


In [13]:
import os, sys
print("CWD:", os.getcwd())
print("Python:", sys.executable)

CWD: /Users/anthonylewallen/Programming/ai_engineering/generatively_pretrained_transformer/src
Python: /opt/anaconda3/bin/python


In [14]:
string_to_int = {ch:i for i, ch in enumerate(chars) }
int_to_string = {i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print(encode("Leo Tolstoi"))
print(decode(encode("Leo Tolstoi")))

[40, 61, 71, 1, 48, 71, 68, 75, 76, 71, 65]
Leo Tolstoi


In [15]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([3227518]) torch.int64
tensor([48, 64, 61,  1, 44, 74, 71, 66, 61, 59, 76,  1, 35, 77, 76, 61, 70, 58,
        61, 74, 63,  1, 61, 30, 71, 71, 67,  1, 71, 62,  1, 51, 57, 74,  1, 57,
        70, 60,  1, 44, 61, 57, 59, 61, 11,  1, 58, 81,  1, 40, 61, 71,  1, 48,
        71, 68, 75, 76, 71, 81,  0,  0, 48, 64, 65, 75,  1, 61, 30, 71, 71, 67,
         1, 65, 75,  1, 62, 71, 74,  1, 76, 64, 61,  1, 77, 75, 61,  1, 71, 62,
         1, 57, 70, 81, 71, 70, 61,  1, 57, 70, 81, 79, 64, 61, 74, 61,  1, 65,
        70,  1, 76, 64, 61,  1, 49, 70, 65, 76, 61, 60,  1, 47, 76, 57, 76, 61,
        75,  1, 57, 70, 60,  0, 69, 71, 75, 76,  1, 71, 76, 64, 61, 74,  1, 72,
        57, 74, 76, 75,  1, 71, 62,  1, 76, 64, 61,  1, 79, 71, 74, 68, 60,  1,
        57, 76,  1, 70, 71,  1, 59, 71, 75, 76,  1, 57, 70, 60,  1, 79, 65, 76,
        64,  1, 57, 68, 69, 71, 75, 76,  1, 70, 71,  1, 74, 61, 75, 76, 74, 65,
        59, 76, 65, 71, 70, 75,  0, 79, 64, 57, 76, 75, 71, 61, 78, 61, 74, 13,
      

In [16]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([48, 64, 61,  1, 44, 74, 71, 66, 61])

In [17]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context.tolist()} the target: {target.item()}")

when input is [48] the target: 64
when input is [48, 64] the target: 61
when input is [48, 64, 61] the target: 1
when input is [48, 64, 61, 1] the target: 44
when input is [48, 64, 61, 1, 44] the target: 74
when input is [48, 64, 61, 1, 44, 74] the target: 71
when input is [48, 64, 61, 1, 44, 74, 71] the target: 66
when input is [48, 64, 61, 1, 44, 74, 71, 66] the target: 61


In [18]:
torch.manual_seed(1369)
batch_size = 4
block_size = 8

def get_batch(split):
    
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs")
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target: {target}")

inputs
torch.Size([4, 8])
tensor([[ 76,  65,  78,  61,   1,  72,  74,  71],
        [ 57,  76,   1,  76,  64,  61,  74,  61],
        [ 62,  76,  61,  74,   1,  79,  65,  75],
        [  1,  70,  65,  59,  61,  74,  11, 110]])
torch.Size([4, 8])
tensor([[ 65,  78,  61,   1,  72,  74,  71,  72],
        [ 76,   1,  76,  64,  61,  74,  61,  11],
        [ 76,  61,  74,   1,  79,  65,  75,  60],
        [ 70,  65,  59,  61,  74,  11, 110,   1]])
----
When input is [76] the target: 65
When input is [76, 65] the target: 78
When input is [76, 65, 78] the target: 61
When input is [76, 65, 78, 61] the target: 1
When input is [76, 65, 78, 61, 1] the target: 72
When input is [76, 65, 78, 61, 1, 72] the target: 74
When input is [76, 65, 78, 61, 1, 72, 74] the target: 71
When input is [76, 65, 78, 61, 1, 72, 74, 71] the target: 72
When input is [57] the target: 76
When input is [57, 76] the target: 1
When input is [57, 76, 1] the target: 76
When input is [57, 76, 1, 76] the target: 64
When input i

In [19]:
print(xb)

tensor([[ 76,  65,  78,  61,   1,  72,  74,  71],
        [ 57,  76,   1,  76,  64,  61,  74,  61],
        [ 62,  76,  61,  74,   1,  79,  65,  75],
        [  1,  70,  65,  59,  61,  74,  11, 110]])


In [33]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1369)

class MyLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:    
            B, T, C = logits.shape 
            logits = logits.view(B*T, C) # Explicitly detail to a 2D B, C, T shape for cross_entropy
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss  # loss is 5.3340, initial guestimate is a bit off
    
    
    def generate(self, idx, max_new_tokens):
        # the idx is a (B, T) array of indices i the current context
        for _ in range(max_new_tokens):
            # Predictions
            logits, loss = self(idx)
            # Last time step
            logits = logits[:, -1, :]
            # Softmax
            probs = F.softmax(logits, dim=-1)
            # Get a Sample
            idx_next = torch.multinomial(probs, num_samples=1)
            # Append the sample to the sequence
            idx = torch.cat((idx, idx_next), dim=1)
            
        return idx
            
    

m = MyLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# zero is new line character
 
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 111])
tensor(5.3340, grad_fn=<NllLossBackward0>)

Á-Yéqèêx1;O68-EèäägäAE2eu,zKOgwe .l04E*HbIr. ëR,j‘î%uu0Zô33ôo[gGczYXLR-Y=nï8îlaaäBSRCoÁq'LcOHa"I .Vq


In [34]:
# a pytorch optimization object
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [56]:
batch_size = 32
for steps in range(100000):
    
    # sample data for training
    xb, yb = get_batch('train')
    
    # evaluate loss
    # We notice that the loss eventually minimizes at approx 2.25 - 2.49
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())
    

2.3886566162109375


In [ ]:
# Sentences with some formed words but mostly gibberish are printed.
# Basically the tokens are not cohesive or talking to each other.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


“We bothf

mpof uprerust tllowhilyan m teleld ollkn.
fe
verng he wad. ame stheve The se wim Pin hore o ayo in y, nthtemaew
v ssorn! wineoulin t.
outhengarepe s, t; unthersthepenon cavaneenowhectid XXXI?... antinowin wll’sca hialit cee mayoas. aue am a w’sa augherenoldesef pa hitheat aze fon’l herevanasof alangroly ana pis as a arag?” Whelly p sangl waistomeee toiserríl t wimpppous ftal ored ede’?”
“cke ato bly
wo
mprminomevewad ete ng tsas jowisewer burínd, someng ay anaca gha og cieace ryovong

